In [ ]:
import ray 
import numpy as np
import logging
import time

import warnings
warnings.filterwarnings('ignore') 

Stime = time.perf_counter()

# Initialize the logger
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

# Function that runs a computation on CPU
@ray.remote(num_cpus=1, runtime_env={"pip": ["minio", "tblib"], "env_vars":{
                                                                                                            "HTTP_PROXY": "http://10.78.90.46:80", 
                                                                                                            "HTTPS_PROXY": "http://10.78.90.46:80",
                                                                                                            "http_proxy": "http://10.78.90.46:80", 
                                                                                                            "https_proxy": "http://10.78.90.46:80"}})
def perform_cpu_task():
    try:
        # Check if CPU is available and being utilized
        if ray.get_gpu_ids():
            # Perform metric multiplication on the CPU using NumPy
            a = np.array([[1, 2], [3, 4]], dtype=np.float32)
            b = np.array([[5, 6], [7, 8]], dtype=np.float32)
            result = np.matmul(a, b)
            return result
        else:
            logger.error("GPU not available.")
            return None
    except Exception as e:
        logger.error(f"Error in GPU task: {e}")
        return None
    
@ray.remote(num_cpus=1, runtime_env={"pip": ["minio", "tblib"], "env_vars":{
                                                                                                            "HTTP_PROXY": "http://10.78.90.46:80", 
                                                                                                            "HTTPS_PROXY": "http://10.78.90.46:80",
                                                                                                            "http_proxy": "http://10.78.90.46:80", 
                                                                                                            "https_proxy": "http://10.78.90.46:80"}})
def create_matrix(size):
    return np.random.normal(size=size)

@ray.remote(num_cpus=1, runtime_env={"pip": ["minio", "tblib"], "env_vars":{
                                                                                                            "HTTP_PROXY": "http://10.78.90.46:80", 
                                                                                                            "HTTPS_PROXY": "http://10.78.90.46:80",
                                                                                                            "http_proxy": "http://10.78.90.46:80", 
                                                                                                            "https_proxy": "http://10.78.90.46:80"}})
def multiply_matrices(x, y):
    return np.dot(x, y)

@ray.remote(num_cpus=1, runtime_env={"pip": ["minio", "tblib"], "env_vars":{
                                                                                                            "HTTP_PROXY": "http://10.78.90.46:80", 
                                                                                                            "HTTPS_PROXY": "http://10.78.90.46:80",
                                                                                                            "http_proxy": "http://10.78.90.46:80", 
                                                                                                            "https_proxy": "http://10.78.90.46:80"}})
def sum_matrices(x, y):
    return np.add(x, y)

print(f"Finished in {time.perf_counter()-Stime:.2f}")

def main():
    try:
        # Initialize Ray
        ray.init(address="ray://kuberay-head-svc.kuberay:10001", ignore_reinit_error=True, runtime_env={"pip": ["minio", "tblib"], 
                                                                                                        "env_vars":{
                                                                                                            "HTTP_PROXY": "http://10.78.90.46:80", 
                                                                                                            "HTTPS_PROXY": "http://10.78.90.46:80",
                                                                                                            "http_proxy": "http://10.78.90.46:80", 
                                                                                                            "https_proxy": "http://10.78.90.46:80"}})
        
        print("=============:", ray.cluster_resources())

        # Call the remote function on GPU
        m1 = create_matrix.remote([10000, 10000])
        m2 = create_matrix.remote([10000, 10000])
        m3 = create_matrix.remote([10000, 10000])
        m4 = create_matrix.remote([10000, 10000])

        m12 = multiply_matrices.remote(m1, m2)
        m34 = multiply_matrices.remote(m3, m4)

        a12_34 =  sum_matrices.remote(m12, m34)

        ## Results
        MM = ray.get(a12_34)
        result = ray.get(perform_cpu_task.remote())
        
        if result is not None:
            logger.info("Metric Multiplication Result on CPU:")
            logger.info(result)
            logger.info(MM)
        else:
            logger.error("CPU task failed.")
    except Exception as e:
        logger.error(f"Error in main: {e}")
    finally:
        # Shutdown Ray gracefully
        ray.shutdown()

if __name__ == "__main__":
    main()

2023-08-04 15:57:51,025	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.7.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-08-04 15:57:51,032	INFO client_builder.py:237 -- Passing the following kwargs to ray.init() on the server: ignore_reinit_error


Finished in 0.00
=============: {'node:__internal_head__': 1.0, 'node:10.224.2.8': 1.0, 'CPU': 3.0, 'node:10.224.0.178': 1.0, 'object_store_memory': 4678641254.0, 'memory': 16000000000.0}


(raylet, ip=10.224.0.178) Spilled 4577 MiB, 6 objects, write throughput 87 MiB/s.
